**INSTALL PACKAGES AND IMPORT THEM**

In [333]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.utils.validation import column_or_1d
from sklearn.feature_selection import VarianceThreshold
from sklearn.naive_bayes import GaussianNB       ### Because continuous data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, train_test_split
import sklearn.neighbors as nb
from sklearn.model_selection import cross_val_predict  
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import sklearn.model_selection as cv 

**UTILS**

In [334]:

def load_data():
  global df
  try: 
    df = pd.read_csv("../data/preprocessed.csv")
  except FileNotFoundError:
    from google.colab import drive
    drive.mount('/content/drive')
    df=pd.read_csv('/content/drive/MyDrive/MD/Laboratori/Practica 2/data/smoking.csv')

def byclass(report):
    report = [x.split(' ') for x in report.split('\n')]
    header = ['Class Name']+[x for x in report[0] if x!='']
    values = []
    for row in report[1:-5]:
        row = [value for value in row if value!='']
        if row!=[]:
            values.append(row)
    df = pd.DataFrame(data = values, columns = header)
    return df

def accuracy(report):
  report = [x.split('      ') for x in report.split('\n')]
  df = pd.DataFrame(report)
  selected_rows = df.iloc[[6, 7]]
  selected_rows = selected_rows.drop([4, 5], axis=1)
  display(selected_rows)
  return 

def accuracy2(report):
  report = [x.split('      ') for x in report.split('\n')]
  report[6].pop(2)
  df = pd.DataFrame(report)
  selected_rows = df.iloc[[6, 7],[0,1,2]]
  display(selected_rows)
  return

def prettyReport(report):
    df = byclass(report)
    
    display(df)
    print("\n")
    df
    accuracy(report)
    return

def prettyReport2(report):
    df = byclass(report)
    display(df)
    print("\n")
    df
    accuracy2(report)
    return

In [335]:
load_data()

In [336]:
df.head()

,age,height(cm),weight(kg),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,fasting blood sugar,Cholesterol,triglyceride,HDL,hemoglobin,Urine protein,serum creatinine,ALT,Gtp,dental caries,tartar,smoking
0,40,155,60,1.2,1.0,1.0,1.0,114.0,94.0,215.0,82.0,73.0,12.9,1.0,0.7,19.0,27.0,False,True,False
1,40,160,60,0.8,0.6,1.0,1.0,119.0,130.0,192.0,115.0,42.0,12.7,1.0,0.6,19.0,18.0,False,True,False
2,55,170,60,0.8,0.8,1.0,1.0,138.0,89.0,242.0,182.0,55.0,15.8,1.0,1.0,16.0,22.0,False,False,True
3,40,165,70,1.5,1.5,1.0,1.0,100.0,96.0,322.0,254.0,45.0,14.7,1.0,1.0,26.0,18.0,False,True,False
4,40,155,60,1.0,1.0,1.0,1.0,120.0,80.0,184.0,74.0,62.0,12.5,1.0,0.6,14.0,22.0,False,False,False


## NAIVE BAYES METHOD

**Preprocessed Dataset without balancing and mean encoding**

In [337]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['smoking'], axis=1)
Y = df['smoking']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [338]:
#Prediction
clf = GaussianNB()

clf.fit(X_train,Y_train)

y_pred = clf.predict(X_test)
 

print(confusion_matrix(Y_test, y_pred))
print(accuracy_score(Y_test, y_pred))
prettyReport(classification_report(Y_test, y_pred))



[[4869 1894]
 [1331 2698]]
0.7011675315048184


,Class Name,precision,recall,f1-score,support
0,False,0.79,0.72,0.75,6763
1,True,0.59,0.67,0.63,4029


,0,1,2,3
6,macro avg,0.69,0.69,0.69 10792
7,weighted avg,0.71,0.70,0.70 10792


**Cross Validation**

In [339]:
cv = StratifiedKFold(n_splits=10, random_state=30, shuffle =True) 

clf = GaussianNB()
y_np = Y.to_numpy()
y_np = y_np.ravel()
cv_scores = cross_val_score(clf,X=X,y=y_np,cv=cv)
np.mean(cv_scores)

predicted = cross_val_predict(clf, X=X, y=y_np,  cv=cv)  

print(confusion_matrix(y_np, predicted))
print(accuracy_score(y_np, predicted))
prettyReport2(classification_report(y_np, predicted))

[[24485  9315]
 [ 6871 13285]]
0.7000148268959894


,Class Name,precision,recall,f1-score,support
0,False,0.78,0.72,0.75,33800
1,True,0.59,0.66,0.62,20156


,0,1,2
6,macro avg,0.68,0.69 53956
7,weighted avg,0.71,0.70
